In [2]:
import cv2
import numpy as np
from keras.models import model_from_json
import keras.utils as image

def detect_emotions_realtime():
    # Load emotion recognition model
    model = model_from_json(open("model\\emotion_model.json", "r").read())
    model.load_weights('model\\emotion_model.h5')
    emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

    # Load face detection cascade classifier
    path_haarcascade = 'haarcascades\\haarcascade_frontalface_default.xml'
    face_haar_cascade = cv2.CascadeClassifier(path_haarcascade)

    # Open webcam
    cap = cv2.VideoCapture(0)

    # Set window size
    cv2.namedWindow('Emotion Detection', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Emotion Detection', 800, 600)

    while True:
        # Capture frame from webcam
        ret, frame = cap.read()

        if not ret:
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = face_haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Process each detected face
        for (x, y, w, h) in faces:
            # Draw bounding box around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), thickness=2)

            # Extract ROI (Region of Interest) - face area from grayscale image
            roi_gray = gray[y:y + h, x:x + w]

            # Resize ROI to match the input size of emotion recognition model
            roi_gray = cv2.resize(roi_gray, (48, 48))

            # Convert ROI to array
            img_pixels = image.img_to_array(roi_gray)

            # Expand dimensions of the array
            img_pixels = np.expand_dims(img_pixels, axis=0)

            # Normalize pixel values
            img_pixels /= 255

            # Predict emotion using the model
            predictions = model.predict(img_pixels)

            # Find the emotion with the highest probability
            max_index = np.argmax(predictions[0])
            predicted_emotion = emotions[max_index]

            # Display the predicted emotion on the frame
            cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Display the frame
        cv2.imshow('Emotion Detection', frame)

        # Check for 'q' key press to exit
        if cv2.waitKey(1) == ord('q'):
            break

    # Release the webcam and close windows
    cap.release()
    cv2.destroyAllWindows()

# Gọi hàm nhận diện khuôn mặt và cảm xúc trong thời gian thực
detect_emotions_realtime()


1/1 [==============================] - 0s 16ms/step
